# Natural Language Processing - Sentiment Analysis Spring 2023

#### Team: Priyanka Agarwal (CIS 5190), Karleena Rybacki (CIS 5190), Praveen Raj Uma Maheswari Shyam Sundar (CIS 5190). 

##### Project Mentor TA: Yiming Huang

## **Milestone-2**

In [ ]:
# Some imports that you might need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import nltk
import os

In [ ]:
!pip install --upgrade --no-cache-dir gdown
if not os.path.exists("Reviews.csv"):
    !gdown 1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk

In [ ]:
# Remeber to change the path here for the corresponding files you need
df = pd.read_csv('Reviews.csv')
print(df.shape)
df_head = df.head(1000)
print(df.shape)

In [ ]:
df = df[['Text', 'Score']]
df.dropna(inplace=True)

In [ ]:
#Maping the Scores to binary labels:
df['Label'] = df['Score'].apply(lambda x: 'positive' if x >= 4 else 'negative' if x <= 2 else 'neutral')


In [ ]:
#Dataset Splitting:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
#Evaluation Metrics:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
#Approach One - Logistic Regression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


vectorizer = CountVectorizer()


train_vectors = vectorizer.fit_transform(train_df['Text'])


clf = LogisticRegression(max_iter=80)
clf.fit(train_vectors, train_df['Label'])


val_vectors = vectorizer.transform(val_df['Text'])
val_predictions = clf.predict(val_vectors)


y_test = val_df['Label']
y_pred_lr = val_predictions

lr_metrics = evaluate(y_test, y_pred_lr)
print("Logistic Regression Baseline Metrics: ", lr_metrics)


In [ ]:
#Approach Two - Support Vector Machine (SVM)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC




model_svm = LinearSVC()
model_svm.fit(train_vectors, train_df['Label'])


y_pred_svm = model_svm.predict(val_vectors)


svm_metrics = evaluate(y_test, y_pred_svm)
print("SVM Metrics: ", svm_metrics)


## **Milestone-3**

CNN Implementation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, GlobalMaxPooling1D
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import keras



#Preparing the data for training
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_df['Text'])


X_train = tokenizer.texts_to_sequences(train_df['Text'])
X_train = pad_sequences(X_train, maxlen=100)
y_train = pd.get_dummies(train_df['Label']).values


X_val = tokenizer.texts_to_sequences(val_df['Text'])
X_val = pad_sequences(X_val, maxlen=100)
y_val = pd.get_dummies(val_df['Label']).values


X_test = tokenizer.texts_to_sequences(test_df['Text'])
X_test = pad_sequences(X_test, maxlen=100)
y_test = pd.get_dummies(test_df['Label']).values


#Building the CNN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=100))
model.add(Conv1D(filters=32, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))


model.add(Conv1D(filters=64, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(GlobalMaxPooling1D())


model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))


optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])




#Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))


#Evaluating the model
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
scores = evaluate(y_test, y_pred)
print(scores)



Epoch 1/10
5685/5685 [==============================] - 222s 39ms/step - loss: 0.3436 - accuracy: 0.8728 - val_loss: 0.2889 - val_accuracy: 0.8942
Epoch 2/10
5685/5685 [==============================] - 227s 40ms/step - loss: 0.2560 - accuracy: 0.9068 - val_loss: 0.2746 - val_accuracy: 0.9025
Epoch 3/10
5685/5685 [==============================] - 218s 38ms/step - loss: 0.2136 - accuracy: 0.9239 - val_loss: 0.2670 - val_accuracy: 0.9067
Epoch 4/10
5685/5685 [==============================] - 216s 38ms/step - loss: 0.1809 - accuracy: 0.9365 - val_loss: 0.2746 - val_accuracy: 0.9073
Epoch 5/10
5685/5685 [==============================] - 209s 37ms/step - loss: 0.1543 - accuracy: 0.9466 - val_loss: 0.3087 - val_accuracy: 0.9094
Epoch 6/10
5685/5685 [==============================] - 196s 35ms/step - loss: 0.1332 - accuracy: 0.9540 - val_loss: 0.3092 - val_accuracy: 0.9044
Epoch 7/10
5685/5685 [==============================] - 199s 35ms/step - loss: 0.1155 - accuracy: 0.9605 - val_loss: 0

Custom **LSTM** Implementation

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import nltk
import os
#from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from keras.layers import LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

os.system("pip3 install --upgrade --no-cache-dir gdown")
if not os.path.exists("Reviews.csv"):
    os.system("gdown 1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk")



df = pd.read_csv('Reviews.csv')
print(df.shape)
df_head = df.head(1000)
print(df.shape)



df = df[['Text', 'Score']]


df.dropna(inplace=True)


#Maping the Scores to binary labels:
df['Label'] = df['Score'].apply(lambda x: 'positive' if x >= 4 else 'negative' if x <= 2 else 'neutral')



#Dataset Splitting:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


#Evaluation Metrics:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluate(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



##Tokenizing the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['Text'])
train_sequences = tokenizer.texts_to_sequences(train_df['Text'])
val_sequences = tokenizer.texts_to_sequences(val_df['Text'])


# Padding sequences
max_sequence_len = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_len, padding='post', truncating='post')
val_sequences = pad_sequences(val_sequences, maxlen=max_sequence_len, padding='post', truncating='post')


#Integer encoding of labels
label_encoder = LabelEncoder()
label_encoder.fit(train_df['Label'])
train_labels = label_encoder.transform(train_df['Label'])
val_labels = label_encoder.transform(val_df['Label'])


#Defining the LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(len(tokenizer.word_index)+1, 100, input_length=max_sequence_len))
lstm_model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Defining callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('lstm_model.h5', save_best_only=True)


#Fitting the LSTM model
lstm_model.fit(train_sequences, train_labels, validation_data=(val_sequences, val_labels), epochs=10, batch_size=64)


#Loading the best model
lstm_model.load_weights('lstm_model.h5')

val_pred = lstm_model.predict_classes(val_sequences)
val_pred = val_pred.ravel()
precision = precision_score(val_labels, val_pred)
recall = recall_score(val_labels, val_pred)
f1 = f1_score(val_labels, val_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)


(568454, 10)
(568454, 10)
Epoch 1/10
  21/5685 [..............................] - ETA: 42:43 - loss: -0.6822 - accuracy: 0.0707

KeyboardInterrupt: ignored

**LSTM** with **GloVe** Word Embeddings

In [ ]:
#Imports
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [ ]:
#Downloading the dataset and pre-trained embeddings (GloVe)
os.system("pip3 install --upgrade --no-cache-dir gdown")
if not os.path.exists("Reviews.csv"):
    os.system("gdown 1_kLSwiRYtiXF7h9V1FlTqTapOHiYU5Mk")

if not os.path.exists("glove.6B.100d.txt"):
    os.system("gdown https://nlp.stanford.edu/data/glove.6B.zip")
    os.system("unzip glove.6B.zip glove.6B.100d.txt")

#Loading the dataset
df = pd.read_csv('Reviews.csv')
print(df.shape)

#Selecting a subset of the data (to speed up training)
df = df.sample(frac=0.1, random_state=42)

#Extracting only the relevant columns
df = df[['Text', 'Score']]
df.dropna(inplace=True)

#Mapping the Scores to binary labels:
df['Label'] = df['Score'].apply(lambda x: 'positive' if x >= 4 else 'negative' if x <= 2 else 'neutral')

#Dataset Splittinig
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


In [ ]:
#Tokenizing the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['Text'])
train_sequences = tokenizer.texts_to_sequences(train_df['Text'])
val_sequences = tokenizer.texts_to_sequences(val_df['Text'])

#Padding the sequencess
max_sequence_len = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_len, padding='post', truncating='post')
val_sequences = pad_sequences(val_sequences, maxlen=max_sequence_len, padding='post', truncating='post')

#Integer encoding the labels
label_encoder = LabelEncoder()
label_encoder.fit(train_df['Label'])
train_labels = label_encoder.transform(train_df['Label'])
val_labels = label_encoder.transform(val_df['Label'])

#Loading pre-trained word embeddings - GloVe Word Embeddings
embedding_dim = 100
embeddings_index = {}
with open("glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs

#Creating an embedding matrix for our vocabulary
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
#Defining the LSTM model with pre-trained word embeddings

lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_len, trainable=False))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dense(3, activation='softmax'))


#Compiling the model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


lstm_model.summary()

#Training the model
batch_size = 64
epochs = 10

history = lstm_model.fit(train_sequences, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_sequences, val_labels))

#Evaluating
test_sequences = tokenizer.texts_to_sequences(test_df['Text'])
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_len, padding='post', truncating='post')
test_labels = label_encoder.transform(test_df['Label'])
test_loss, test_acc = lstm_model.evaluate(test_sequences, test_labels)
#test_loss, test_acc = lstm_model.evaluate(test_sequences, test_labels, verbose=2)

print('Test Accuracy:', test_acc)

# Evaluate the model on the test data
#test_sequences = tokenizer.texts_to_sequences(test_df['Text'])
#test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_len, padding='post', truncating='post')
#test_labels = label_encoder.transform(test_df['Label'])


# Make predictions on the test data
#test_predictions = lstm_model.predict_classes(test_sequences)
test_predictions = np.argmax(lstm_model.predict(test_sequences), axis=-1)


# Convert the integer labels back to original labels
test_predictions = label_encoder.inverse_transform(test_predictions)

# Print the evaluation metrics
from sklearn.metrics import classification_report

print("Test Accuracy: {:.2f}%".format(test_acc * 100))
print(classification_report(test_df['Label'], test_predictions))